In [35]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:

import requests
from shapely.geometry import box


def get_bb(place_name):
    base_url = "https://nominatim.openstreetmap.org"
    format_out = "json"
    limit = 10

    # Construct the API request URL
    url = f"{base_url}/search?city={place_name}&format={format_out}&limit={limit}"

    # Send the API request
    response = requests.get(url).json()

    results = []
    for result in response:
        if 'boundingbox' in result:
            bounding_box = result['boundingbox']
            if len(bounding_box) == 4:
                min_lon, max_lon, min_lat, max_lat = map(float, bounding_box)
                results.append({"name": f"{result['display_name']}", "bbox": box(min_lon, min_lat, max_lon, max_lat) })

    return results

In [5]:
place_name = "el Prat de Llobregat"
results = get_bb(place_name)

if len(results) == 1:
    # Only one result, access the bounding box directly
    print(results[0]["bbox"])
else:
    # Multiple results, iterate over the dictionary
    for dict in results:
        print(dict)
        

{'name': 'el Prat de Llobregat, Baix Llobregat, Barcelona, Catalunya, 08820, España', 'bbox': <POLYGON ((41.348 2.057, 41.348 2.168, 41.277 2.168, 41.277 2.057, 41.348 2....>}
{'name': 'el Prat de Llobregat, Baix Llobregat, Barcelona, Catalunya, 08820, España', 'bbox': <POLYGON ((41.348 2.057, 41.348 2.168, 41.277 2.168, 41.277 2.057, 41.348 2....>}


In [122]:

import geopandas as gpd

def get_box_or_gdfs_by_place_name(place_name):
    results = get_bb(place_name)
    gdfs = []
    if len(results) == 1:
        # Only one result, access the bounding box directly
        return results[0]["bbox"]
    else:
        # Multiple results, iterate over the dictionary
        for dict in results:
            gdf = gpd.GeoDataFrame(geometry=[dict["bbox"]], crs=4326)
            gdfs.append({"name": dict["name"], "gdf": gdf})
        return gdfs

In [123]:
get_box_or_gdfs_by_place_name("el Prat de Llobregat")

[{'name': 'el Prat de Llobregat, Baix Llobregat, Barcelona, Catalunya, 08820, España',
  'gdf':                                             geometry
  0  POLYGON ((41.34834 2.05733, 41.34834 2.16762, ...},
 {'name': 'el Prat de Llobregat, Baix Llobregat, Barcelona, Catalunya, 08820, España',
  'gdf':                                             geometry
  0  POLYGON ((41.34834 2.05733, 41.34834 2.16762, ...},
 {'name': 'el Prat de Llobregat, Passatge de la Ribera, el Prat de Llobregat, Baix Llobregat, Barcelona, Catalunya, 08820, España',
  'gdf':                                             geometry
  0  POLYGON ((41.33589 2.08426, 41.33589 2.09426, ...},
 {'name': 'el Prat de Llobregat, Passatge de la Ribera, el Prat de Llobregat, Baix Llobregat, Barcelona, Catalunya, 08820, España',
  'gdf':                                             geometry
  0  POLYGON ((41.33098 2.08922, 41.33098 2.08932, ...},
 {'name': 'el Prat de Llobregat, Camí de la Ribera, Vinyets i Molí Vell, el Prat de Ll